# Grid Search Hyper-parameters

In this notebook, we'll cover how to define hyper parameters for your grid search, and how to determine the proper grid search strategy for you. We will **not** actually fit a grid search in this notebook, we'll simply introduce how to setup the hyper parameters for either approach.

Scikit-learn defines two types of grid search:
1. __GridSearchCV__
2. __RandomizedSearchCV__

In [1]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

## What exactly is a grid search?

In a grid search, we can fit a model with a selection of various different hyper parameter combinations to find one that best-fits our model. Rarely will you have enough business domain knowledge or data understanding to fit the perfect model from intuition. A grid search allows us to search a "hyper parameter search space" for the best combination of model hyper params.

### Exhaustive searches

Search through every possible combination of provided hyper params.

* Scale combinatorially
* Look at EVERY combination of hyper parameters

<img alt="Exhaustive search" src="img/exhaustive-search.png" width="50%"/>

### Randomized searches

Randomly sample combinations of hyper parameters for a set number of iterations.

* Generally faster
* Randomly select hyper parameter combinations

<img alt="Exhaustive search" src="img/random-search.png" width="50%"/>

## *Which should you use?...*

It depends... but generally favor the randomized search unless time is no factor (keep your business requirements in mind)

# Problem setup

We'll use the scikit-learn native "breast cancer" dataset:

Breast Cancer Wisconsin (Diagnostic) Database
=============================================

Notes
-----
Data Set Characteristics:
    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, field
        13 is Radius SE, field 23 is Worst Radius.

        - class:
                - WDBC-Malignant
                - WDBC-Benign

    :Summary Statistics:

    ===================================== ====== ======
                                           Min    Max
    ===================================== ====== ======
    radius (mean):                        6.981  28.11
    texture (mean):                       9.71   39.28
    perimeter (mean):                     43.79  188.5
    area (mean):                          143.5  2501.0
    smoothness (mean):                    0.053  0.163
    compactness (mean):                   0.019  0.345
    concavity (mean):                     0.0    0.427
    concave points (mean):                0.0    0.201
    symmetry (mean):                      0.106  0.304
    fractal dimension (mean):             0.05   0.097
    radius (standard error):              0.112  2.873
    texture (standard error):             0.36   4.885
    perimeter (standard error):           0.757  21.98
    area (standard error):                6.802  542.2
    smoothness (standard error):          0.002  0.031
    compactness (standard error):         0.002  0.135
    concavity (standard error):           0.0    0.396
    concave points (standard error):      0.0    0.053
    symmetry (standard error):            0.008  0.079
    fractal dimension (standard error):   0.001  0.03
    radius (worst):                       7.93   36.04
    texture (worst):                      12.02  49.54
    perimeter (worst):                    50.41  251.2
    area (worst):                         185.2  4254.0
    smoothness (worst):                   0.071  0.223
    compactness (worst):                  0.027  1.058
    concavity (worst):                    0.0    1.252
    concave points (worst):               0.0    0.291
    symmetry (worst):                     0.156  0.664
    fractal dimension (worst):            0.055  0.208
    ===================================== ====== ======

    :Missing Attribute Values: None

    :Class Distribution: 212 - Malignant, 357 - Benign

    :Creator:  Dr. William H. Wolberg, W. Nick Street, Olvi L. Mangasarian

    :Donor: Nick Street

    :Date: November, 1995

This is a copy of UCI ML Breast Cancer Wisconsin (Diagnostic) datasets.
https://goo.gl/U2Uwz2

Features are computed from a digitized image of a fine needle
aspirate (FNA) of a breast mass.  They describe
characteristics of the cell nuclei present in the image.

Separating plane described above was obtained using
Multisurface Method-Tree (MSM-T) [K. P. Bennett, "Decision Tree
Construction Via Linear Programming." Proceedings of the 4th
Midwest Artificial Intelligence and Cognitive Science Society,
pp. 97-101, 1992], a classification method which uses linear
programming to construct a decision tree.  Relevant features
were selected using an exhaustive search in the space of 1-4
features and 1-3 separating planes.

The actual linear program used to obtain the separating plane
in the 3-dimensional space is that described in:
[K. P. Bennett and O. L. Mangasarian: "Robust Linear
Programming Discrimination of Two Linearly Inseparable Sets",
Optimization Methods and Software 1, 1992, 23-34].

This database is also available through the UW CS ftp server:

ftp ftp.cs.wisc.edu
cd math-prog/cpo-dataset/machine-learn/WDBC/

References
----------
   - W.N. Street, W.H. Wolberg and O.L. Mangasarian. Nuclear feature extraction 
     for breast tumor diagnosis. IS&T/SPIE 1993 International Symposium on 
     Electronic Imaging: Science and Technology, volume 1905, pages 861-870,
     San Jose, CA, 1993.
   - O.L. Mangasarian, W.N. Street and W.H. Wolberg. Breast cancer diagnosis and 
     prognosis via linear programming. Operations Research, 43(4), pages 570-577, 
     July-August 1995.
   - W.H. Wolberg, W.N. Street, and O.L. Mangasarian. Machine learning techniques
     to diagnose breast cancer from fine-needle aspirates. Cancer Letters 77 (1994) 
     163-171.

In [7]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import pandas as pd

bc = load_breast_cancer()
X, y = pd.DataFrame.from_records(bc.data, columns=bc.feature_names), bc.target

random_state = 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

print("n_samples=%i, n_features=%i" % (X_train.shape))
X_train.head()

n_samples=455, n_features=30


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
68,9.029,17.33,58.79,250.5,0.10660,0.14130,0.31300,0.04375,0.2111,0.08046,...,10.31,22.65,65.50,324.7,0.14820,0.43650,1.25200,0.17500,0.4228,0.11750
181,21.090,26.57,142.70,1311.0,0.11410,0.28320,0.24870,0.14960,0.2395,0.07398,...,26.68,33.48,176.50,2089.0,0.14910,0.75840,0.67800,0.29030,0.4098,0.12840
63,9.173,13.86,59.20,260.9,0.07721,0.08751,0.05988,0.02180,0.2341,0.06963,...,10.01,19.23,65.59,310.1,0.09836,0.16780,0.13970,0.05087,0.3282,0.08490
248,10.650,25.22,68.01,347.0,0.09657,0.07234,0.02379,0.01615,0.1897,0.06329,...,12.25,35.19,77.98,455.7,0.14990,0.13980,0.11250,0.06136,0.3409,0.08147
60,10.170,14.88,64.55,311.9,0.11340,0.08061,0.01084,0.01290,0.2743,0.06960,...,11.02,17.45,69.86,368.6,0.12750,0.09866,0.02168,0.02579,0.3557,0.08020


# Defining (reasonable) search parameters

1. Understand your algorithm and how it's different knobs affect how it works
2. Realize there are some ranges that don't make sense
3. Know your data and how different hyper-parameters might affect bias/variance

### Setup WITHOUT a pipeline:

While not recommended (and very rare that you'd not have a pre-processing pipeline), grid searches do not require a pipeline. Hyper parameters are defined simply with a dictionary keyed by the hyper parameter names you'd like to tune. Any non-specified hyper parameter will use the algorithm's default value.

In [10]:
from sklearn.ensemble import RandomForestClassifier

hyper_params= {
    'n_estimators': [10, 20, 50, 100, 1000, 10000],  # is this reasonable?
    'max_depth': [10, 15, 25, 1000],  # is this reasonable?
    'min_samples_split': [1, 5, 50, 500]  # is this reasonable?
}

### Setup WITH a pipeline:

Setup with a pipeline requires a bit of extra care on the naming side. Note a slight difference: all keys in the hyper parameters dictionary must before with the NAME of the stage in the pipeline separated by a double underscore from the name of the hyper parameter.

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest

# first make your pipe
pipe = Pipeline([
        ('skb', SelectKBest()),
        ('clf', RandomForestClassifier(random_state=random_state))  # only non-variable hyper params here
    ])

# now set hyperparams
hyper_params = {
    'skb__k': [1, 2, 3, 5, 10, 20, 'all'],
    'clf__n_estimators': [10, 20, 50, 100, 1000, 10000],  # is this reasonable?
    'clf__max_depth': [10, 15, 25, 1000],  # is this reasonable?
    'clf__min_samples_split': [1, 5, 50, 500]  # is this reasonable?
}

# What about random searches?

What we've shown so far will work for either exhaustive OR random searches, but we can get a bit more clever in our random search spaces. Note that in random searches, any iterable search space provided is subject to uniform sampling with replacement.

In [ ]:
from scipy.stats import uniform, randint

hyper_params = {
    'skb__k': [5, 10, 20, 'all'],
    'clf__n_estimators': randint(50, 250),
    'clf__max_depth': randint(5, 25),
    'clf__min_samples_split': randint(2, 15),
    'clf__min_samples_leaf': uniform(0.05, 0.1)  # mean, std
}

## In a future notebook: *fitting*